In [1]:

import os

from utils.path_utils import project_root

import torch

# Load data
from adatime.load_data import load_data
from adatime.configs.get_configs import get_configs


# Training

In [2]:
import pandas as pd
from adatime.utils import AverageMeter, fix_randomness, save_checkpoint, calculate_metrics, calculate_risks, \
    append_results_to_tables, add_mean_std_table, save_tables_to_file
import collections
from adatime.log_utils.log_experiment import starting_logs
from adatime.da.models import get_backbone_class
from adatime.da.algorithms import get_algorithm_class

data_path = os.path.join(project_root(), 'data', 'adatime_datasets', 'HHAR', 'HHAR_SA')
print(f"Data loading from {data_path}")

dataset_name = 'HHAR'
dataset_configs, hparams = get_configs(dataset_name)

results_columns = ["scenario", "run", "acc", "f1_score", "auroc"]
risks_columns = ["scenario", "run", "src_risk", "few_shot_risk", "trg_risk"]

# table with metrics
table_results = pd.DataFrame(columns=results_columns)
# table with risks
table_risks = pd.DataFrame(columns=risks_columns)

device = 'cuda:0'
num_runs = 1
for src_id, trg_id in dataset_configs.scenarios:
    
    for run_id in range(num_runs):
        
        fix_randomness(run_id)
        
        # Loading Data
        src_train, src_test, tgt_train, tgt_test, few_shot_dl_5 = load_data(
            data_path=data_path, src_id=src_id, trg_id=trg_id, dataset_configs=dataset_configs, 
            hparams=hparams)
        
        # Setting up DA algorithm
        da_algorithm_name = 'HoMM'
        da_algorithm = get_algorithm_class(da_algorithm_name)
        
        # Setting up backbone
        backbone_name = 'CNN'
        da_backbone = get_backbone_class(backbone_name) 
        
        # Initializing algorithm
        algorithm = da_algorithm(da_backbone, dataset_configs, hparams, device, 
                                 da_algorithm_name)
        algorithm.to(device)
        
        # Train the domain adaptation algorithm
        home_path =  os.getcwd()
        save_dir = 'experiment_logs'
        experiment_description = dataset_name
        exp_name = 'testing_da_framework'
        run_description = f"{da_algorithm_name}_{exp_name}"
        
        exp_log_dir = os.path.join(home_path, save_dir, experiment_description, f"{run_description}")
        
        logger, scenario_log_dir = starting_logs(dataset_name, da_algorithm_name, 
                                                 exp_log_dir, src_id, trg_id, run_id)
        
        loss_avg_meters = collections.defaultdict(lambda: AverageMeter())
        last_model, best_model = algorithm.update(src_train, tgt_train, loss_avg_meters, logger)
        
        # Save checkpoint
        save_checkpoint(home_path, scenario_log_dir, last_model, best_model)
        
        # Calculate risks and metrics
        metrics = calculate_metrics(algorithm, tgt_test, dataset_configs, device)
        risks = calculate_risks(algorithm, src_test, tgt_test, few_shot_dl_5, device)
        
        # Append results to tables
        scenario = f"{src_id}_to_{trg_id}"
        table_results = append_results_to_tables(table_results, scenario, run_id, metrics)
        table_risks = append_results_to_tables(table_risks, scenario, run_id, risks)
    
# Calculate and append mean and std to tables
print("add_mean_std_table for table")
table_results = add_mean_std_table(table_results, results_columns)

print("add_mean_std_table for risks")
table_risks = add_mean_std_table(table_risks, risks_columns)

# Save tables to file if needed
save_tables_to_file(exp_log_dir, table_results, 'results')
save_tables_to_file(exp_log_dir, table_risks, 'risks')
    

Data loading from /localscratch/neeresh/data/physionet2019/data/adatime_datasets/HHAR/HHAR_SA
Dataset: HHAR
Method:  HoMM
Source: 0 ---> Target: 6
Run ID: 0
[Epoch : 1/40]
Total_loss	: 0.1901
HoMM_loss	: 0.0024
Src_cls_loss	: 0.9332
-------------------------------------
[Epoch : 2/40]
Total_loss	: 0.1504
HoMM_loss	: 0.0027
Src_cls_loss	: 0.7343
-------------------------------------
[Epoch : 3/40]
Total_loss	: 0.1274
HoMM_loss	: 0.0028
Src_cls_loss	: 0.6193
-------------------------------------
[Epoch : 4/40]
Total_loss	: 0.1107
HoMM_loss	: 0.0029
Src_cls_loss	: 0.5361
-------------------------------------
[Epoch : 5/40]
Total_loss	: 0.0991
HoMM_loss	: 0.0030
Src_cls_loss	: 0.4782
-------------------------------------
[Epoch : 6/40]
Total_loss	: 0.0897
HoMM_loss	: 0.0030
Src_cls_loss	: 0.4310
-------------------------------------
[Epoch : 7/40]
Total_loss	: 0.0826
HoMM_loss	: 0.0030
Src_cls_loss	: 0.3959
-------------------------------------
[Epoch : 8/40]
Total_loss	: 0.0768
HoMM_loss	

/home/nperla_umassd_edu/Documents/SepsisEarlyPrediction/adatime/utils.py:131: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  table = pd.concat([table, results_df], ignore_index=True)


Dataset: HHAR
Method:  HoMM
Source: 1 ---> Target: 6
Run ID: 0
[Epoch : 1/40]
Total_loss	: 0.1404
HoMM_loss	: 0.0041
Src_cls_loss	: 0.6777
-------------------------------------
[Epoch : 2/40]
Total_loss	: 0.1024
HoMM_loss	: 0.0035
Src_cls_loss	: 0.4918
-------------------------------------
[Epoch : 3/40]
Total_loss	: 0.0825
HoMM_loss	: 0.0032
Src_cls_loss	: 0.3942
-------------------------------------
[Epoch : 4/40]
Total_loss	: 0.0697
HoMM_loss	: 0.0032
Src_cls_loss	: 0.3308
-------------------------------------
[Epoch : 5/40]
Total_loss	: 0.0614
HoMM_loss	: 0.0032
Src_cls_loss	: 0.2899
-------------------------------------
[Epoch : 6/40]
Total_loss	: 0.0551
HoMM_loss	: 0.0031
Src_cls_loss	: 0.2585
-------------------------------------
[Epoch : 7/40]
Total_loss	: 0.0501
HoMM_loss	: 0.0031
Src_cls_loss	: 0.2338
-------------------------------------
[Epoch : 8/40]
Total_loss	: 0.0461
HoMM_loss	: 0.0030
Src_cls_loss	: 0.2142
-------------------------------------
[Epoch : 9/40]
Total_loss

# Testing

In [3]:
import numpy as np
from adatime.utils import load_checkpoint, evaluate

# Results dataframes
last_results = pd.DataFrame(columns=results_columns)
best_results = pd.DataFrame(columns=results_columns)

# Cross-domain scenarios
for src_id, trg_id in dataset_configs.scenarios:
    for run_id in range(num_runs):
        # fixing random seed
        fix_randomness(run_id)
        
        # Logging
        scenario_log_dir = os.path.join(exp_log_dir, src_id + "_to_" + trg_id + "_run_" + str(run_id))
        loss_avg_meters = collections.defaultdict(lambda: AverageMeter())
        
        # Load data
        src_train, src_test, tgt_train, tgt_test, few_shot_dl_5 = load_data(
        data_path=data_path, src_id=src_id, trg_id=trg_id, dataset_configs=dataset_configs, 
        hparams=hparams)
        
        # Setting up DA algorithm
        da_algorithm_name = 'HoMM'
        da_algorithm = get_algorithm_class(da_algorithm_name)
        
        # Setting up backbone
        backbone_name = 'CNN'
        da_backbone = get_backbone_class(backbone_name) 
        
        # Initializing algorithm
        algorithm = da_algorithm(da_backbone, dataset_configs, hparams, device, 
                                 da_algorithm_name)
        algorithm.to(device)
        
        # Loading checkpoint
        last_chk, best_chk = load_checkpoint(scenario_log_dir)
        
        # Testing the last model
        algorithm.network.load_state_dict(last_chk)
        loss, full_preds, full_labels = evaluate(algorithm, tgt_test, device)
        last_metrics = calculate_metrics(algorithm, tgt_test, dataset_configs, device)
        last_results = append_results_to_tables(last_results, f"{src_id}_to_{trg_id}", run_id,
                                                     last_metrics)
        
        # Testing the best model
        algorithm.network.load_state_dict(best_chk)
        loss, full_preds, full_labels = evaluate(algorithm, tgt_test, device)
        best_metrics = calculate_metrics(algorithm, tgt_test, dataset_configs, device)
        
        # Append results to tables
        best_results = append_results_to_tables(best_results, f"{src_id}_to_{trg_id}", run_id,
                                                     best_metrics)
        
last_scenario_mean_std = last_results.groupby('scenario')[['acc', 'f1_score', 'auroc']].agg(['mean', 'std'])
best_scenario_mean_std = best_results.groupby('scenario')[['acc', 'f1_score', 'auroc']].agg(['mean', 'std'])

# Save tables to file if needed
save_tables_to_file(exp_log_dir, last_scenario_mean_std, 'last_results')
save_tables_to_file(exp_log_dir, best_scenario_mean_std, 'best_results')

# printing summary 
summary_last = {metric: np.mean(last_results[metric]) for metric in results_columns[2:]}
summary_best = {metric: np.mean(best_results[metric]) for metric in results_columns[2:]}
for summary_name, summary in [('Last', summary_last), ('Best', summary_best)]:
    for key, val in summary.items():
        print(f'{summary_name}: {key}\t: {val:2.4f}')
        
    # break
    

/home/nperla_umassd_edu/Documents/SepsisEarlyPrediction/adatime/utils.py:131: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  table = pd.concat([table, results_df], ignore_index=True)


Last: acc	: 0.7563
Last: f1_score	: 0.7304
Last: auroc	: 0.0000
Best: acc	: 0.7520
Best: f1_score	: 0.7302
Best: auroc	: 0.0000
